In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch
import torch.nn as nn
import torch.cuda as cuda
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt

## Loading the training data

In [ ]:
training_dataset = torchvision.datasets.ImageFolder('../input/ift6135h19/trainset/trainset', transform = transforms.ToTensor())
train_data, valid_data = torch.utils.data.random_split(training_dataset, (18000, 1998))
train_loader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size = 16)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle = True, batch_size = 16)

## CNN Architecture
Inspired by https://towardsdatascience.com/image-classifier-cats-vs-dogs-with-convolutional-neural-networks-cnns-and-google-colabs-4e9af21ae7a8

In [ ]:
class CatsVsDogsNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(2)
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv1.weight)
        self.conv2 = nn.Conv2d(32, 32, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv2.weight)
        self.conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv3.weight)
        self.conv4 = nn.Conv2d(64, 64, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv4.weight)
        self.conv5 = nn.Conv2d(64, 128, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv5.weight)
        self.conv6 = nn.Conv2d(128, 128, 3, padding = 1)
        torch.nn.init.xavier_uniform_(self.conv6.weight)
        
        self.dense1 = nn.Linear(8192, 1000)
        self.dense2 = nn.Linear(1000, 2)
    
    def forward(self, x):
        #1st conv layer
        x = self.conv1(x)
        x = self.relu(x)
        
        #2nd conv layer
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #3rd conv layer
        x = self.conv3(x)
        x = self.relu(x)

        #4th conv layer
        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #5th conv layer
        x = self.conv5(x)
        x = self.relu(x)
        
        #6th conv layer
        x = self.conv6(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = x.view(-1, 8192)
        
        #1st fully connected layer
        x = self.dense1(x)
        x = self.relu(x)
        
        #2nd fully connected layer
        x = self.dense2(x)
        
        return x
        

## Creating objects, loss function & optimizer

In [ ]:
cats_dogs_net = CatsVsDogsNet()

if cuda.is_available:
    cats_dogs_net.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cats_dogs_net.parameters(), lr = 0.08)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,4,gamma=0.8)

## Training phase

In [ ]:
train_loss = []
train_accuracy = []

valid_loss = []
valid_accuracy = []

for epoch in range(10):
    epoch_loss = 0           #keeps the count of the total loss per epoch
    accurate_predictions = 0 #keeps the count of the number of accurate predictions in the current epoch
    scheduler.step()
    
    for i, data in enumerate(train_loader):
        inputs, targets = data
        
        if cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
            
        optimizer.zero_grad()
        outputs = cats_dogs_net(inputs)
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        #Evaluation of accurate predictions
        _, prediction = torch.max(outputs.data, 1)
        accurate_predictions += (prediction == targets.data).sum().cpu().numpy().astype('float64')
    
    train_loss.append(epoch_loss)
    train_accuracy.append(100 * accurate_predictions/len(train_loader.dataset))
    
    epoch_loss = 0
    accurate_predictions = 0
    
    for i, data in enumerate(valid_loader):
        inputs, targets = data
        
        if cuda.is_available():
            inputs = inputs.cuda()
            targets = targets.cuda()
        
        #evaluation of the loss function
        outputs = cats_dogs_net(inputs)
        loss = criterion(outputs, targets)
        epoch_loss += loss.item()
        
        #evaluation of accurate predictions
        _, prediction = torch.max(outputs,1)
        accurate_predictions += (prediction == targets.data).sum().cpu().numpy().astype('float64')
    
    valid_loss.append(epoch_loss)
    valid_accuracy.append(100 * accurate_predictions/len(valid_loader.dataset))
    
    print('Epoch: ', epoch+1, 'train_accuracy: ', train_accuracy[-1], 'train_loss: ', train_loss[-1])
    print('Epoch: ', epoch+1, 'valid_accuracy: ', valid_accuracy[-1], 'valid_loss: ', valid_loss[-1])
        
        
        
    

## Loading test data

In [ ]:
test_dataset = torchvision.datasets.ImageFolder('../input/testset-numerical-order/testset/testset', transform = transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 2)

In [ ]:
import numpy as np
import pandas as pd
import kaggle

## Classifying the test set

In [ ]:
id_row = np.arange(4999) + 1 
prediction_list = []
for i, data in enumerate(test_loader):
    inputs, _ = data
    
    if cuda.is_available():
            inputs = inputs.cuda()
    
    outputs = cats_dogs_net(inputs)
    predictions = torch.max(outputs.data,1)[1]
    prediction_list.extend(list(predictions.cpu().numpy()))

In [ ]:
prediction_strings = ['Cat' if x==0 else 'Dog' for x in prediction_list]

In [ ]:
submission = pd.DataFrame(
    {'id': id_row,
     'label': prediction_strings,
    })

In [ ]:
submission.to_csv('submission.csv', index = False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_error = 100 - np.asarray(train_accuracy)
validation_error = 100 - np.asarray(valid_accuracy)

training_loss = np.asarray(train_loss)/18000
validation_loss = np.asarray(valid_loss)/1998

epochs = np.arange(10)+1

In [ ]:
plt.plot(epochs,training_error, label = 'Training error')
plt.plot(epochs, validation_error, label = 'Validation error')
plt.legend(loc='upper right')
plt.title('Classification error per epoch; training vs validation')
plt.xlabel('Epoch')
plt.ylabel('Error per epoch (%)')


In [ ]:
plt.plot(epochs,training_loss, label = 'Training loss')
plt.plot(epochs, validation_loss, label = 'Validation loss')
plt.legend(loc='upper right')
plt.title('Loss per epoch; training vs validation')
plt.xlabel('Epoch')
plt.ylabel('Average loss per epoch')
